In [2]:
download_dir = './arxiv_downloads'

In [3]:
import os
files = os.listdir(download_dir)
files[:3]

['2503.20488v1.Adaptive_Local_Clustering_over_Attributed_Graphs.pdf',
 '2503.20491v1.VPO__Aligning_Text_to_Video_Generation_Models_with_Prompt_Optimization.pdf',
 '2503.20492v1.Towards_Efficient_and_General_Purpose_Few_Shot_Misclassification_Detection_for_Vision_Language_Models.pdf']

In [4]:
from pypdf import PdfReader
reader = PdfReader(os.path.join(download_dir, files[0]))
len(reader.pages), reader.pages[0].extract_text()[:1000]

(21,
 'Adaptive Local Clustering over Attributed Graphs\nTechnical Report\nHaoran Zheng\nHong Kong Baptist University\nHong Kong SAR, China\ncshrzheng@comp.hkbu.edu.hk\nRenchi Yang\nHong Kong Baptist University\nHong Kong SAR, China\nrenchi@hkbu.edu.hk\nJianliang Xu\nHong Kong Baptist University\nHong Kong SAR, China\nxujl@hkbu.edu.hk\nAbstract—Given a graph G and a seed node vs, the objective\nof local graph clustering (LGC) is to identify a subgraph Cs ∈ G\n(a.k.a. local cluster) surrounding vs in time roughly linear with\nthe size of Cs. This approach yields personalized clusters without\nneeding to access the entire graph, which makes it highly suitable\nfor numerous applications involving large graphs. However, most\nexisting solutions merely rely on the topological connectivity\nbetween nodes in G, rendering them vulnerable to missing or\nnoisy links that are commonly present in real-world graphs.\nTo address this issue, this paper resorts to leveraging the\ncomplementary nature 

In [22]:
from rolling.embedding import Corpus
import tqdm
corpora = []
for file in tqdm.tqdm(files[:10]):
    reader = PdfReader(os.path.join(download_dir, file))
    text = ''
    for page in reader.pages:
        text += page.extract_text()
    corpora.append(Corpus(file, text, n=128))

100%|██████████| 10/10 [01:30<00:00,  9.04s/it]


In [23]:
import random
def random_color():
    return (random.random(), random.random(), random.random())  # RGB tuple

In [24]:
import umap
import numpy as np

# Collect all embeddings and corresponding PDF labels
all_embeddings = np.vstack([emb for corpus in corpora for emb in corpus.embeddings])
labels = [corpus.title for corpus in corpora for _ in corpus.embeddings]

# Reduce to 2D using UMAP
reducer = umap.UMAP(n_components=3, random_state=42)
low_dim_embeddings = reducer.fit_transform(all_embeddings)
color_map = {title: random_color() for title in labels}

d:\rolling_embedding\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
d:\rolling_embedding\.venv\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [25]:
import matplotlib
matplotlib.use("TkAgg") 

In [26]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # For 3D plotting
import mplcursors

# Your 3D plotting code
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')

start = 0
scatter_plots = []
for corpus in corpora:
    num_points = len(corpus.embeddings)
    subset = low_dim_embeddings[start:start+num_points]
    
    # Scatter plot for each PDF in 3D
    scatter = ax.scatter(subset[:, 0], subset[:, 1], subset[:, 2], 
                         color=color_map[corpus.title], label=corpus.title, alpha=0.6)
    
    # Add the scatter plot and corresponding text parts for hover
    scatter_plots.append((scatter, corpus.parts))  # Store the scatter and the corresponding text parts
    
    start += num_points

# Add the legend and labels
ax.legend(loc="upper center", bbox_to_anchor=(0.5, -0.1))
ax.set_title("3D Embeddings Visualization")
ax.set_xlabel("UMAP Dim 1")
ax.set_ylabel("UMAP Dim 2")
ax.set_zlabel("UMAP Dim 3")

# Use mplcursors to display text on hover
cursor = mplcursors.cursor(hover=True)

# Define the callback to show and hide annotations
@cursor.connect("add")
def on_add(sel):
    # Get the index of the hovered point
    index = sel.index
    # Find the corresponding part of the text
    part_text = corpus.parts[index]
    # Set the annotation text to the text part
    sel.annotation.set_text(part_text)

@cursor.connect("remove")
def on_remove(sel):
    # Hide the annotation when the mouse moves off
    sel.annotation.set_text("")

plt.show()


d:\rolling_embedding\.venv\Lib\site-packages\mplcursors\_pick_info.py:503: UserWarning: 3d coordinates not supported yet
  warnings.warn("3d coordinates not supported yet")
Traceback (most recent call last):
  File "d:\rolling_embedding\.venv\Lib\site-packages\matplotlib\cbook.py", line 361, in process
    func(*args, **kwargs)
  File "d:\rolling_embedding\.venv\Lib\site-packages\mplcursors\_mplcursors.py", line 580, in _on_hover_motion_notify
    self._on_select_event(event)
  File "d:\rolling_embedding\.venv\Lib\site-packages\mplcursors\_mplcursors.py", line 628, in _on_select_event
    self.add_selection(pi)
  File "d:\rolling_embedding\.venv\Lib\site-packages\mplcursors\_mplcursors.py", line 411, in add_selection
    cb(sel)
  File "C:\Users\Tobias\AppData\Local\Temp\ipykernel_14268\177993679.py", line 40, in on_add
    part_text = corpus.parts[index]
                ~~~~~~~~~~~~^^^^^^^
IndexError: list index out of range
Traceback (most recent call last):
  File "d:\rolling_embedd